In [6]:
from sqlalchemy import *
import pandas as pd
import numpy as np
import random

pd.options.display.max_rows = 99999
pd.options.display.max_columns = 99999

In [7]:
# setup engine to archbox psql database
arch_engine = create_engine("postgresql://<user>:<pass>192.168.0.2:5432/sampalytics")

In [8]:
# get list of tickers in our db
tickers = arch_engine.engine.execute('SELECT DISTINCT ticker from stocks.minute').fetchall()
# convert to list, ie get the tickers out of the tuples
tickers = [x[0] for x in tickers]

In [9]:
outcome = 'AMD'
n_predictors = 100

In [10]:
def prune_list(outcome, ticks, n_predictors):
    # remove the outcome var so we dont randomly select it and create duplicates
    ticks = [x for x in ticks if x not in outcome] # .remove causes pointer issue
    # convert list of tickers to df so we can index
    df = pd.DataFrame(ticks)
    # randomly select the rows we want to take
    rows = random.sample(range(len(ticks)), n_predictors)
    # then select the rows subset
    df = df.iloc[rows]
    return [outcome] + list(df[0])

In [11]:
tickers = prune_list(outcome=outcome, ticks=tickers, n_predictors=n_predictors)

In [12]:
# custom header
full_df = pd.DataFrame(columns=['dtg'])
# predictors = ['open', 'high', 'low', 'close', 'volume']
predictors = ['close', 'volume'] # try less to save memory
i = 1
for ticker in tickers:
    header = ['dtg'] + [x + '_' + str(ticker) for x in predictors]
    select=', '.join(['dtg'] + predictors)
    # d = arch_engine.engine.execute("SELECT DISTINCT dtg, open, high, low, close, volume from stocks.minute WHERE ticker='{}'".format(ticker)).fetchall()
    d = arch_engine.engine.execute("SELECT DISTINCT {} from stocks.minute WHERE ticker='{}'".format(select, ticker)).fetchall()
    d = pd.DataFrame(d, columns=header).dropna()
    print('Loading: {}: {}'.format(ticker, i))
    i = i + 1
    full_df = pd.merge(full_df, d, on='dtg', how='outer')

Loading: AMD: 1
Loading: ANAT: 2
Loading: NTRA: 3
Loading: KMI: 4
Loading: IROQ: 5
Loading: CCD: 6
Loading: FAD: 7
Loading: IXY: 8
Loading: FBIZ: 9
Loading: INDY: 10
Loading: CCLP: 11
Loading: CPLP: 12
Loading: ROBT: 13
Loading: CNSL: 14
Loading: MNST: 15
Loading: FINX: 16
Loading: DNLI: 17
Loading: CARS: 18
Loading: KTOV: 19
Loading: MARA: 20
Loading: GSBC: 21
Loading: RFEM: 22
Loading: CODA: 23
Loading: ARRS: 24
Loading: SPA: 25
Loading: TC: 26
Loading: ETX: 27
Loading: PVTL: 28
Loading: DOVA: 29
Loading: OCSL: 30
Loading: AGS: 31
Loading: CYAD: 32
Loading: AMRB: 33
Loading: FORR: 34
Loading: PFIN: 35
Loading: HMI: 36
Loading: TUSA: 37
Loading: RBBN: 38
Loading: CERC: 39
Loading: FEMB: 40
Loading: CCCR: 41
Loading: CUTR: 42
Loading: CENT: 43
Loading: KERX: 44
Loading: IIJI: 45
Loading: HTHT: 46
Loading: JCS: 47
Loading: ILMN: 48
Loading: IFEU: 49
Loading: BIIB: 50
Loading: DTYS: 51
Loading: CREE: 52
Loading: CARA: 53
Loading: SCON: 54
Loading: LMRKN: 55
Loading: ENTG: 56
Loading: CHK

In [ ]:
# ie. row 1 is Jan 1, row 2 in Jan 2.
full_df.sort_values('dtg', inplace=True)
# and do a forward fill
full_df.fillna(method='ffill', inplace=True)
# then drop the NA
full_df.dropna(inplace=True)
# rename dtg to date for TS package
full_df.rename(columns={'dtg':'Date'}, inplace=True)

In [13]:
full_df.to_csv('/mnt/ubudata/projects/data/AMD_plus_100.csv',index=None)

In [20]:
# continue in main-notebook of forecast engine